In this notebook, I have tried stacking concept from an article in Anlaytics Vidhya(https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-for-ensemble-models/, This article details about various ensemble techniques. ) to predict whether a customer gets loan approved based on his/her details which we use as features.Also tried some frequent ML models like Logistic Regression, Random Forest Classifier, Support Vector Classifier and Gradiant Boosting. Surprisingly Logistic Regression achieved Highest score in terms of predicting unseen data.

In [164]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore') 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [25]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [26]:
categorical_cols=list(train.select_dtypes(include=object).columns)
categorical_cols.remove('Loan_ID')

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null object
Married              614 non-null object
Dependents           614 non-null object
Education            614 non-null object
Self_Employed        614 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           592 non-null float64
Loan_Amount_Term     600 non-null float64
Credit_History       564 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
Loan_ID              367 non-null object
Gender               367 non-null object
Married              367 non-null object
Dependents           367 non-null object
Education            367 non-null object
Self_Employed        367 non-null object
ApplicantIncome      367 non-null int64
CoapplicantIncome    367 non-null int64
LoanAmount           362 non-null float64
Loan_Amount_Term     361 non-null float64
Credit_History       338 non-null float64
Property_Area        367 non-null object
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


In [27]:
for col in categorical_cols:
    print(col)
    print(train[col].value_counts())

Gender
Male      489
Female    112
Name: Gender, dtype: int64
Married
Yes    398
No     213
Name: Married, dtype: int64
Dependents
0     345
1     102
2     101
3+     51
Name: Dependents, dtype: int64
Education
Graduate        480
Not Graduate    134
Name: Education, dtype: int64
Self_Employed
No     500
Yes     82
Name: Self_Employed, dtype: int64
Property_Area
Semiurban    233
Urban        202
Rural        179
Name: Property_Area, dtype: int64
Loan_Status
Y    422
N    192
Name: Loan_Status, dtype: int64


In [28]:
train.replace('3+',3,inplace=True)
test.replace('3+',3,inplace=True)

In [40]:
for col in categorical_cols[:-1]+['Loan_Amount_Term','Credit_History']:
    train[col].fillna(train[col].mode()[0],inplace=True)
    test[col].fillna(train[col].mode()[0],inplace=True)

In [41]:
train['LoanAmount'].fillna(train['LoanAmount'].mean(),inplace=True)
test['LoanAmount'].fillna(train['LoanAmount'].mean(),inplace=True)

In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
Loan_ID              614 non-null object
Gender               614 non-null object
Married              614 non-null object
Dependents           614 non-null object
Education            614 non-null object
Self_Employed        614 non-null object
ApplicantIncome      614 non-null int64
CoapplicantIncome    614 non-null float64
LoanAmount           614 non-null float64
Loan_Amount_Term     614 non-null float64
Credit_History       614 non-null float64
Property_Area        614 non-null object
Loan_Status          614 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 62.4+ KB


In [44]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 12 columns):
Loan_ID              367 non-null object
Gender               367 non-null object
Married              367 non-null object
Dependents           367 non-null object
Education            367 non-null object
Self_Employed        367 non-null object
ApplicantIncome      367 non-null int64
CoapplicantIncome    367 non-null int64
LoanAmount           367 non-null float64
Loan_Amount_Term     367 non-null float64
Credit_History       367 non-null float64
Property_Area        367 non-null object
dtypes: float64(3), int64(2), object(7)
memory usage: 34.5+ KB


In [45]:
cols_to_dummies=['Gender','Married','Education','Self_Employed','Property_Area','Loan_Status']
dummies=pd.get_dummies(train[cols_to_dummies])
cols_to_drop=['Gender_Female','Married_No','Education_Not Graduate','Self_Employed_No','Property_Area_Rural','Loan_Status_N']
dummies.drop(cols_to_drop,axis=1,inplace=True)

In [46]:
train_df=pd.concat([train.drop(cols_to_dummies,axis=1),dummies],axis=1)
train_df.head()

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Education_Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
0,LP001002,0,5849,0.0,146.412162,360.0,1.0,1,0,1,0,0,1,1
1,LP001003,1,4583,1508.0,128.000000,360.0,1.0,1,1,1,0,0,0,0
2,LP001005,0,3000,0.0,66.000000,360.0,1.0,1,1,1,1,0,1,1
3,LP001006,0,2583,2358.0,120.000000,360.0,1.0,1,1,0,0,0,1,1
4,LP001008,0,6000,0.0,141.000000,360.0,1.0,1,0,1,0,0,1,1


In [48]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 14 columns):
Loan_ID                    614 non-null object
Dependents                 614 non-null object
ApplicantIncome            614 non-null int64
CoapplicantIncome          614 non-null float64
LoanAmount                 614 non-null float64
Loan_Amount_Term           614 non-null float64
Credit_History             614 non-null float64
Gender_Male                614 non-null uint8
Married_Yes                614 non-null uint8
Education_Graduate         614 non-null uint8
Self_Employed_Yes          614 non-null uint8
Property_Area_Semiurban    614 non-null uint8
Property_Area_Urban        614 non-null uint8
Loan_Status_Y              614 non-null uint8
dtypes: float64(4), int64(1), object(2), uint8(7)
memory usage: 37.9+ KB


In [54]:
train_df.head()

,Loan_ID,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Education_Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban,Loan_Status_Y
0,LP001002,0,5849,0.0,146.412162,360.0,1.0,1,0,1,0,0,1,1
1,LP001003,1,4583,1508.0,128.000000,360.0,1.0,1,1,1,0,0,0,0
2,LP001005,0,3000,0.0,66.000000,360.0,1.0,1,1,1,1,0,1,1
3,LP001006,0,2583,2358.0,120.000000,360.0,1.0,1,1,0,0,0,1,1
4,LP001008,0,6000,0.0,141.000000,360.0,1.0,1,0,1,0,0,1,1


In [57]:
X=train_df.drop(['Loan_ID','Loan_Status_Y'],axis=1)
y=train_df['Loan_Status_Y']

In [58]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1)

In [128]:
def Stacking(model,train,y,test,n_fold):
    folds=StratifiedKFold(n_splits=n_fold,random_state=1)
    test_pred=np.empty((0,1),float)
    train_pred=np.empty((0,1),float)
    for train_indices,val_indices in folds.split(train,y.values):
        x_train,x_val=train.iloc[train_indices],train.iloc[val_indices]
        y_train,y_val=y.iloc[train_indices],y.iloc[val_indices]

        model.fit(X=x_train,y=y_train)
        train_pred=np.append(train_pred,model.predict(x_val))
    test_pred=np.append(test_pred,model.predict(test))
    return test_pred,train_pred

In [129]:
model1 = DecisionTreeClassifier(random_state=1)

test_pred1 ,train_pred1=Stacking(model=model1,n_fold=10, train=X_train,test=X_test,y=y_train)

train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

In [130]:
model2 = KNeighborsClassifier()

test_pred2 ,train_pred2=Stacking(model=model2,n_fold=10,train=X_train,test=X_test,y=y_train)

train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

In [132]:
df = pd.concat([train_pred1, train_pred2], axis=1)
df_test = pd.concat([test_pred1, test_pred2], axis=1)

model = LogisticRegression(random_state=1)
model.fit(df,y_train)
model.score(df_test,y_test)

0.6688311688311688

In [152]:
lr=LogisticRegression(random_state=4)
lr.fit(X_train,y_train)
lr.score(X_test,y_test)

0.7922077922077922

In [149]:
clf=RandomForestClassifier(n_estimators=5,random_state=1).fit(X_train,y_train)
clf.score(X_test,y_test)

0.7402597402597403

In [162]:
gb=GradientBoostingClassifier(learning_rate=0.01,random_state=2).fit(X_train,y_train)
gb.score(X_test,y_test)

0.7857142857142857

In [165]:
sv_clf=SVC(kernel='linear').fit(X_train,y_train)
sv_clf.score(X_test,y_test)

0.7662337662337663